In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-dataset/fake_news_dataset.csv


# Feature_Extraction_of_Text_Data_Using_Tf_idf_Vectorizer

About the Dataset:

id: unique id for a news article
title: the title of a news article
author: author of the news article
text: the text of the article; could be incomplete
label: a label that marks whether the news article is real or fake: 1: Fake news 0: real News

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/kaggle/input/fake-news-dataset/fake_news_dataset.csv')
     

In [6]:
news_dataset.shape

(20800, 5)

In [7]:
news_dataset.sample(10)

,id,title,author,text,label
13115,13115,Neil Armstrong: Their Ships Were Far Superior ...,Arnold Monteverde (noreply@blogger.com),Everyone is familiar with the broadcast images...,1
17635,17635,Obama Ban: Religious Foreign Aid,NaN,"Obama Ban: Religious Foreign Aid November 01, ...",1
14106,14106,Russia to beef up western defenses with S-400'...,Jafe Arnoldski (noreply@blogger.com),"November 21, 2016 - Fort Russ News - RIANovos...",1
16913,16913,Coming Ice Age/ Consciousness & Afterlife,NaN,Coming Ice Age/ Consciousness & Afterlife Co...,1
19735,19735,National Data | October Jobs— Immigrant Workin...,Edwin S. Rubenstein,- < National Data | October Jobs— Immigrant Wo...,1
14325,14325,"Along the Autism Spectrum, a Path Through Camp...",Jan Hoffman,"BOWLING GREEN, Ky. — Crosby J. Gardner has ...",0
19127,19127,"Ferrero zu Kinderarbeitsvorwürfen: ""Aber das s...",noreply@blogger.com (Der Postillon),— The Sun (@TheSun) 23. November 2016 Laut Fer...,1
18177,18177,"Trump: Presidents Obama, Clinton Were ’Outplay...",Pam Key,In a preview clip of an interview set to air o...,0
19987,19987,BREAKING! — New shattering Wikileaks info impl...,Admin,"November 7, 2016 at 6:55 pm \nFlopot, \nYes, I...",1
10208,10208,Comment on Robert De Niro to Produce Documenta...,quietsch,Home / #Solutions / Robert De Niro to Produce ...,1


In [8]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
news_dataset=news_dataset.fillna('')

In [10]:
news_dataset.sample(10)

,id,title,author,text,label
8508,8508,"Trump Budget Guru, Long a Fiscal Hawk, Now Has...",Matt Flegenheimer and Alan Rappeport,WASHINGTON — Making the rounds at the Capit...,0
1751,1751,Life: Happy Halloween! Here Are 7 Thrilling Wa...,,Email All Hallows’ Eve comes just once a year!...,1
3172,3172,Comment on 5 Ways You Can Start Feeling Better...,What are the harsh life lessons everyone shoul...,Share on Facebook Share on Twitter It’s a no b...,1
18739,18739,“Rigged To Deny Trump”: Are They Stealing Thes...,Mac Slavo,Will the 2016 election be rigged?\nThat depend...,1
1929,1929,When Trouble Is on the Way,Gaye Levy,"Backdoor Survival November 1, 2016 \nBack in t...",1
4500,4500,EMP event forthcoming?,vlawde,You have a work bunker? What is it exactly tha...,1
14889,14889,A Transgender Student Won Her Battle. Now It’s...,Anemona Hartocollis,"PALATINE, Ill. — Tall and sylphlike, an ath...",0
806,806,WAR ON THE STREETS OF PARIS: Armed migrants fi...,wmw_admin,"By wmw_admin on November 3, 2016 Nick Gutterid...",1
4436,4436,Mainstream Media Totally Blows It On Retail Sales,Lee Adler,Mainstream Media Totally Blows It On Retail Sa...,1
7990,7990,‘Carrie’ Is Back. Like a Bloody Hand From the ...,Gilbert Cruz,"Released a few days after Halloween in 1976, i...",0


In [11]:
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [12]:
news_dataset.sample(10)

,id,title,author,text,label,content
15304,15304,Fewer Tomatoes in Ketchup? East Europeans Purs...,Rick Lyman,"BRATISLAVA, Slovakia — When Simona Budinska...",0,Rick Lyman Fewer Tomatoes in Ketchup? East Eur...
16145,16145,Bangladeshi Troops Move to End Hostage Standof...,"Julfikar Ali Manik, Geeta Anand and Russell Go...","DHAKA, Bangladesh — Bangladeshi Army troops...",0,"Julfikar Ali Manik, Geeta Anand and Russell Go..."
16391,16391,"World War 3, Information War, and the End of P...",Jafe Arnoldski (noreply@blogger.com),"November 5, 2016 - Fort Russ Exclusive - Inte...",1,Jafe Arnoldski (noreply@blogger.com) World War...
19847,19847,"Dershowitz: Special Counsel Is Wrong Choice, E...",Joel B. Pollak,"Harvard Law School emeritus professor, liberal...",0,Joel B. Pollak Dershowitz: Special Counsel Is ...
10262,10262,Study: Breitbart Dominated Right Wing Coverage...,Allum Bokhari,A study from the Columbia Journalism Review (C...,0,Allum Bokhari Study: Breitbart Dominated Right...
731,731,Russia may run out of patience and respond to ...,,Russia may run out of patience and respond to ...,1,Russia may run out of patience and respond to...
573,573,Le top des recherches Google passe en top des ...,,,1,Le top des recherches Google passe en top des...
19357,19357,Nikonorova: Donbass' UN recognition will be Ki...,Jafe Arnoldski (noreply@blogger.com),"November 17, 2016 - Fort Russ - Antifashist -...",1,Jafe Arnoldski (noreply@blogger.com) Nikonorov...
20676,20676,California Set to Let Public Schools Teach Pri...,Henry Wolff,California Set to Let Public Schools Teach Pri...,1,Henry Wolff California Set to Let Public Schoo...
19403,19403,The Best TV Shows of 2016 - The New York Times,"James Poniewozik, Mike Hale and Neil Genzlinger",lists are stories: They tell the truth by ly...,0,"James Poniewozik, Mike Hale and Neil Genzlinge..."


In [13]:
print(news_dataset['content'])
     

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [14]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [15]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content 
                       if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [18]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [19]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [20]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [21]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [22]:
print(Y)

[1 0 1 ... 0 1 1]


In [23]:
Y.shape

(20800,)

Tf-Idf

In [24]:
# convert the textual data to Feature Vectors
vectorizer = TfidfVectorizer()
     

In [25]:
vectorizer.fit(X)

X = vectorizer.transform(X)

In [26]:
print(X)
     

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420